In [2]:
from binance.client import Client as bnb_client
from datetime import datetime
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from scipy.stats import zscore
import snscrape.modules.twitter as sntwitter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
client = bnb_client(tld='US')

def get_data(symbol,freq,start_ts = '2020-01-01',end_ts='2025-07-15'):
    data = client.get_historical_klines(symbol,freq,start_ts,end_ts)
    columns = ['open_time','open','high','low','close','volume','close_time','quote_volume',
    'num_trades','taker_base_volume','taker_quote_volume','ignore']

    data = pd.DataFrame(data,columns = columns)
    
    # Convert from POSIX timestamp (number of millisecond since jan 1, 1970)
    data['open_time'] = data['open_time'].map(lambda x: datetime.utcfromtimestamp(x/1000))
    data['close_time'] = data['close_time'].map(lambda x: datetime.utcfromtimestamp(x/1000))
    return data

In [4]:
universe = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT', 'SOLUSDT', 'ADAUSDT', 
            'XRPUSDT', 'MATICUSDT', 'DOGEUSDT']
freq = '4h'


px = {}
for x in universe:
    data = get_data(x,freq)
    px[x] = data.set_index('open_time')['close']

px = pd.DataFrame(px).astype(float)
px = px.reindex(pd.date_range(px.index[0],px.index[-1],freq=freq))
ret = px.pct_change()

In [ ]:
!pip install scrapfly-sdk
from scrapfly import ScrapflyClient, ScrapeConfig
from bs4 import BeautifulSoup
import pandas as pd

client = ScrapflyClient(key="YOUR_SCRAPFLY_API_KEY")

# Search Twitter (replace XRP with your term)
query = "XRP"
url = f"https://twitter.com/search?q={query}&src=typed_query&f=live"

result = client.scrape(ScrapeConfig(
    url=url,
    render_js=True  # JS rendering is required for Twitter
))

# Parse tweets from HTML
soup = BeautifulSoup(result.content, "html.parser")
tweets = []

# Note: Twitter structure is constantly changing. Look for tweet containers
for div in soup.find_all("div", attrs={"data-testid": "tweet"}):
    content = div.get_text(separator=" ")
    tweets.append(content)

# Store to DataFrame
df = pd.DataFrame(tweets, columns=["tweet"])
print(df.head())
